## Import

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import pandas as pd
import numpy as np
import re

* load data

In [2]:
data_path = "C:\\Users\\PC\\OneDrive\\문서\\GitHub\\Korea_Investment_and_Securities\\datasets\\"

In [3]:
out_data_path = "C:\\Users\\PC\\OneDrive\\문서\\datasets\\"

In [4]:
sector_df = pd.read_csv(data_path + "sector_df.csv")
stock_01 = pd.read_csv(data_path + "stock_df_01.csv")
tot_df = pd.read_csv(data_path + "tot_df.csv", encoding = "cp949")

In [5]:
tot_df.head()

,Symbol,Name,Industry,IndustryCode
0,TSM,Taiwan Semiconductor Manufacturing Co Ltd ADR,반도체 및 반도체 장비,571010.0
1,V,Visa Inc Class A,소프트웨어 및 IT서비스,572010.0
2,JPM,JPMorgan Chase & Co,은행,551010.0
3,UNH,UnitedHealth Group Inc,헬스케어 업체 및 서비스,561020.0
4,JNJ,Johnson & Johnson,제약,562010.0


* make group df

In [8]:
'''
sector_df = pd.merge(sector_df, stock_01, on = "etf_names", how = "inner")

group_df = sector_df.groupby(["sector", "stock_names"])['etf_names'].count().reset_index()

for idx, sec in enumerate(group_df.sector.unique()):
    temp_df = group_df.loc[group_df.sector == secs]
    temp_df = temp_df.sort_values("etf_names", ascending = False)
    
    if idx == 0 :
        group_df_f = temp_df
    else :
        group_df_f = group_df_f.append(temp_df)

group_df_f.index = [x for x in range(group_df_f.shape[0])]
group_df['stock_names'] = [re.sub(",","",x) for x in group_df.stock_names]
group_df_f.to_csv(data_path + "group_df.csv", index = False)
'''

'\nsector_df = pd.merge(sector_df, stock_01, on = "etf_names", how = "inner")\n\ngroup_df = sector_df.groupby(["sector", "stock_names"])[\'etf_names\'].count().reset_index()\n\nfor idx, sec in enumerate(group_df.sector.unique()):\n    temp_df = group_df.loc[group_df.sector == secs]\n    temp_df = temp_df.sort_values("etf_names", ascending = False)\n    \n    if idx == 0 :\n        group_df_f = temp_df\n    else :\n        group_df_f = group_df_f.append(temp_df)\n\ngroup_df_f.index = [x for x in range(group_df_f.shape[0])]\ngroup_df[\'stock_names\'] = [re.sub(",","",x) for x in group_df.stock_names]\ngroup_df_f.to_csv(data_path + "group_df.csv", index = False)\n'

In [6]:
group_df = pd.read_csv(data_path + "group_df.csv")

* clustering per sector

In [10]:
def print_symbol(name, etf_name, tot_df):
    if name == 'U.S. Dollar':
        return '-'
    # 탐색 key value 선정
    temp_key_value = name.split(" ")
    key_value = temp_key_value[0]

    # TDM 구축
    total_name = []
    total_name.extend(etf_name)
    total_name.extend(list(tot_df.Name))

    tokenizer = CountVectorizer()
    tdm=tokenizer.fit_transform(total_name)

    first_select = [idx for idx, x in enumerate(total_name) if (x == name) or ((x.startswith(key_value)) and idx > len(etf_name))]

    if len(first_select) == 2:
        output_idx = first_select[1] - len(etf_name)
    elif len(first_select) == 1:
        return "-"
    else:
        sims = cosine_similarity(tdm[first_select[0]], tdm[first_select[1:]])
        output_idx = first_select[1:][np.argmax(sims)] - len(etf_name)

#    print("name: ", name,"\nresult:", tot_df.Name[output_idx])
    output = tot_df.iloc[output_idx].Symbol
    return output.split(".")[0]

In [11]:
etf_name = ["Berkshire Hathaway Inc. Class B",
"JPMorgan Chase & Co.", 
"Citigroup Inc.",
"Goldman Sachs Group, Inc."]

In [12]:
symbol = print_symbol(etf_name[3], etf_name, tot_df); symbol

'GS'

In [13]:
group_df.head(6)

,sector,stock_names,etf_names
0,Aerospace & Defense,AAR CORP.,3
1,Aerospace & Defense,HEICO Corporation,3
2,Aerospace & Defense,Northrop Grumman Corporation,3
3,Aerospace & Defense,Moog Inc. Class A,3
4,Aerospace & Defense,Mercury Systems Inc.,3
5,Aerospace & Defense,Maxar Technologies Inc.,3


* clustering

In [14]:
from tqdm import tqdm

In [15]:
symbol_dict = {}

In [16]:
# sector 별로 루프
#for sec in tqdm(group_df.sector.unique()):
for idx, sec in enumerate(group_df.sector.unique()):
    print(idx / group_df.sector.unique().shape[0])
    temp_df = group_df.loc[group_df.sector == sec,:]
    
    # symbol check, 오래걸림
    sec_symbols = []
    for stock in tqdm(temp_df.stock_names):
        symbol = print_symbol(stock, temp_df.stock_names, tot_df)
        if symbol == "-":
            continue
        
        sec_symbols.append(symbol)
    
    symbol_dict[sec] = sec_symbols    

  0%|                                                                                        | 0/61 [00:00<?, ?it/s]

0.0


  0%|                                                                              | 2/1734 [00:00<01:30, 19.23it/s]

0.023255813953488372


  3%|██▍                                                                             | 1/33 [00:00<00:03,  9.61it/s]

...

  0%|                                                                                       | 0/140 [00:00<?, ?it/s]

0.9534883720930233


  0%|                                                                              | 2/6940 [00:00<07:10, 16.13it/s]

0.9767441860465116


100%|███████████████████████████████████████████████████████████████████████████| 6940/6940 [14:31<00:00,  7.97it/s]


In [ ]:
symbol_dict[sec]

In [18]:
import pickle

In [19]:
with open(out_data_path + 'symbol_dict.pickle', 'wb') as f:
    pickle.dump(symbol_dict, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
'''
with open(out_data_path + "symbol_dict.pickle", "rb") as f:
    symbol_dict = pickle.load(f)
'''

* imputation

In [27]:
from sklearn.impute import KNNImputer

In [ ]:
stock_02 = pd.read_csv(out_data_path + "stock_df_02.csv")

'''
select_col = [x for x in stock_02.columns if "_" not in x]
select_col.extend([x for x in stock_02.columns if ("_x" in x) and (".") not in x])
stock_02 = stock_02.loc[:, select_col]
col_names = [x for x in stock_02.columns if "_" not in x]
col_names.extend([x.split("_")[0] for x in stock_02.columns if "_" in x])
stock_02.columns = col_names
stock_02.to_csv(out_data_path + "stock_df_02.csv", index = False)
'''

In [75]:
half_nan  = np.where(stock_02.isnull().sum(axis = 0) <= stock_02.shape[0] * 0.45)[0]

In [77]:
stock_02 = stock_02.iloc[:, half_nan]

In [78]:
imputer = KNNImputer(n_neighbors=5)

In [80]:
imputed_x = imputer.fit_transform(np.array(stock_02.iloc[:, 1:]))

In [81]:
imputed_x.shape

(1831, 4520)

* scaling

In [130]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [131]:
#scaler = StandardScaler()
scaler = MinMaxScaler()

In [132]:
scaled_x= scaler.fit_transform(imputed_x)

In [134]:
scaled_x.shape

(1831, 4520)

In [139]:
stock_02.loc[:, "TSM":] = scaled_x
# stock_02.iloc[:, 1:] = scaled_x

KeyboardInterrupt: 

In [ ]:
stock_02.head()

In [ ]:
stock_02.shape

In [ ]:
#stock_02.to_csv(out_data_path + "scaled_stocks.csv", index = False)
stock_02.to_csv(out_data_path + "mm_scaled_stocks.csv", index = False)

* clustering

In [ ]:
from tslearn.clustering import TimeSeriesKMeans

import matplotlib.pyplot as plt

In [ ]:
symbols = symbol_dict['Artificial Intelligence']
symbols = [x.split(".")[0] for x in symbols]

In [ ]:
cluster_input = np.expand_dims(np.array(stock_02.loc[:, stock_02.columns.isin(symbols)].T), 2)

In [ ]:
n_ = 3

In [ ]:
#model = TimeSeriesKMeans(n_clusters=n_, metric="softdtw", random_state=0, n_jobs = -1,verbose = True)
model = TimeSeriesKMeans(n_clusters=n_, random_state=0, n_jobs = -1,verbose = True)
y_pred = model.fit_predict(cluster_input)

In [ ]:
sz = cluster_input.shape[1]

In [ ]:
for yi in range(n_):
    plt.figure(figsize = (12,6))
    plt.subplot(n_, 1, yi + 1)
    for xx in cluster_input[y_pred == yi]:
        plt.plot(xx.ravel(), "k-", alpha=.2)
    plt.plot(model.cluster_centers_[yi].ravel(), "r-")
    plt.xlim(0, sz)
    plt.ylim(-4, 4)
    plt.text(0.55, 0.85,'Cluster %d' % (yi + 1),
             transform=plt.gca().transAxes)
    if yi == 1:
        plt.title("Soft-DTW $k$-means")

In [ ]:
from sklearn.metrics import silhouette_score

score = silhouette_score(data, preds)
print("Silhouette Score: %.4f" % score)